In [8]:
import pandas as pd
import numpy as np
df = pd.read_csv('NCAA_Tourney_2019_Final.csv')
df.head()

,game_id,team1_seed,team2_seed,team1_dis(KM),team2_dis(KM),team1_pt_school_ncaa,team1_pt_overall_ncaa,team1_pt_school_s16,team1_pt_overall_s16,team1_pt_school_ff,...,team2_arate,team2_opparate,team2_stlrate,team2_oppstlrate,team1_adjtempo,team1_adjoe,team1_adjde,team2_adjtempo,team2_adjoe,team2_adjde
0,2019-1181-1277,1,2,363.386968,756.007983,34,34,24,24,12,...,68.329718,52.331606,0.077123,0.093336,72.2812,120.204,88.2449,67.6316,121.681,90.5738
1,2019-1181-1261,1,3,363.386968,1596.942544,34,34,24,24,12,...,46.052632,44.214373,0.125518,0.090580,72.2812,120.204,88.2449,69.9424,118.620,97.6208
2,2019-1181-1439,1,4,363.386968,349.953580,34,34,24,24,12,...,59.587379,56.376812,0.102672,0.078088,72.2812,120.204,88.2449,63.3196,118.551,94.0834
3,2019-1181-1280,1,5,363.386968,1223.396594,34,34,24,24,12,...,52.030735,50.121359,0.117132,0.085546,72.2812,120.204,88.2449,67.4913,117.496,96.8328
4,2019-1181-1268,1,6,363.386968,13.126713,34,34,24,24,12,...,52.416357,52.016129,0.065536,0.107498,72.2812,120.204,88.2449,65.6956,113.946,93.9660


In [9]:
game_id = df['game_id'].tolist()
df = df.drop(['game_id'],axis = 1)
df.head()

,team1_seed,team2_seed,team1_dis(KM),team2_dis(KM),team1_pt_school_ncaa,team1_pt_overall_ncaa,team1_pt_school_s16,team1_pt_overall_s16,team1_pt_school_ff,team1_pt_overall_ff,...,team2_arate,team2_opparate,team2_stlrate,team2_oppstlrate,team1_adjtempo,team1_adjoe,team1_adjde,team2_adjtempo,team2_adjoe,team2_adjde
0,1,2,363.386968,756.007983,34,34,24,24,12,12,...,68.329718,52.331606,0.077123,0.093336,72.2812,120.204,88.2449,67.6316,121.681,90.5738
1,1,3,363.386968,1596.942544,34,34,24,24,12,12,...,46.052632,44.214373,0.125518,0.090580,72.2812,120.204,88.2449,69.9424,118.620,97.6208
2,1,4,363.386968,349.953580,34,34,24,24,12,12,...,59.587379,56.376812,0.102672,0.078088,72.2812,120.204,88.2449,63.3196,118.551,94.0834
3,1,5,363.386968,1223.396594,34,34,24,24,12,12,...,52.030735,50.121359,0.117132,0.085546,72.2812,120.204,88.2449,67.4913,117.496,96.8328
4,1,6,363.386968,13.126713,34,34,24,24,12,12,...,52.416357,52.016129,0.065536,0.107498,72.2812,120.204,88.2449,65.6956,113.946,93.9660


In [17]:
array = df.values
x = array[:,0:65]
y = array[:,65]
x.shape, y.shape

((2278, 65), (2278,))

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# 标准化
sc = StandardScaler()
x_std = sc.fit_transform(x)
# 主成分分析PCA
pca = PCA(n_components = 'mle',svd_solver='full')
x_pca = pca.fit_transform(x_std)
x_pca.shape

(2278, 64)

In [20]:
# logistic
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=1)
param_grid = [{'penalty':['l1','l2'],
                   'C':[0.01,0.05,0.1,0.5,1,5,10,50,100],
                    'solver':['liblinear'],
                    'multi_class':['ovr']},
                {'penalty':['l2'],
                 'C':[0.01,0.05,0.1,0.5,1,5,10,50,100],
                'solver':['lbfgs'],
                'multi_class':['ovr','multinomial']}]
search = GridSearchCV(lr, param_grid, cv=10)
search.fit(x_pca, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

ValueError: Unknown label type: 'continuous'

In [56]:
from sklearn.metrics import accuracy_score
y_pred = search.predict(x_pca)
accuracy = accuracy_score(y, y_pred)
accuracy

0.7113309352517986

In [57]:
#返回probability 第二列表示该场比赛team1 win的概率
y_prob = search.predict_proba(x_pca)
y_prob

array([[0.47575357, 0.52424643],
       [0.14346756, 0.85653244],
       [0.74854681, 0.25145319],
       ...,
       [0.52404999, 0.47595001],
       [0.62139541, 0.37860459],
       [0.70406904, 0.29593096]])

In [58]:
df['team1_outcome_pred'] = y_pred[:]

In [59]:
team1_win = []
for lst in y_prob:
    p = lst[1]
    team1_win.append(p)
team1_win

[0.5242464346068544,
 0.8565324397738573,
 0.2514531936776493,
 0.08307314735207728,
 0.5324157854235696,
 0.07852881764382405,
 0.5684649179147403,
 0.6382117433549855,
 0.21868778590002544,
 0.5948958866420162,
 0.7301931262254301,
 0.8692180537191887,
 0.5586166775535364,
 0.5473356221241213,
 0.6349540448523152,
 0.5098019543112425,
 0.747303965742961,
 0.42949332564935244,
 0.103644373022071,
 0.17488351133603655,
 0.652553894941101,
 0.21342833492784805,
 0.7636223200128945,
 0.08426444169936077,
 0.68667474341502,
 0.5146456453472839,
 0.1106955240330307,
 0.7336114517507849,
 0.7224050987275826,
 0.6516421314455276,
 0.5315598827605071,
 0.5630408267967167,
 0.30258962523498156,
 0.8006318938087631,
 0.7905203341558723,
 0.30465898788522,
 0.14651983551326575,
 0.5772869832539154,
 0.4446802250850809,
 0.28894490399426837,
 0.2677655920995765,
 0.6994613976355554,
 0.2532015898332835,
 0.8462775092545193,
 0.3061605229207635,
 0.38465326257255755,
 0.33806720608709173,
 0.45678

In [60]:
df['game_id'] = game_id
df['probability'] = team1_win
df.head()

,team1_seed,team2_seed,team1_dis(km),team2_dis(km),team1_pt_school_ncaa,team1_pt_overall_ncaa,team1_pt_school_s16,team1_pt_overall_s16,team1_pt_school_ff,team1_pt_overall_ff,...,team1_adjtempo,team1_adjoe,team1_adjde,team2_adjtempo,team2_adjoe,team2_adjde,team1_outcome,team1_outcome_pred,game_id,probability
0,16,16,1076.682624,932.116689,6,6,0,0,0,0,...,73.8504,99.8665,106.4500,67.7359,98.6102,99.6885,0,1.0,2002-1373-1108,0.524246
1,2,15,969.775258,1058.567916,0,2,0,0,0,0,...,69.9001,111.4954,93.8770,71.2446,96.8669,99.9263,1,1.0,2002-1104-1194,0.856532
2,14,3,1210.852647,513.529264,0,0,0,0,0,0,...,64.7948,105.4534,97.6704,72.8207,117.3877,96.9262,0,0.0,2002-1112-1364,0.251453
3,16,1,339.990955,148.384590,3,3,0,0,0,0,...,67.4639,95.3632,99.9754,75.1685,118.5999,87.7504,0,0.0,2002-1181-1457,0.083073
4,5,12,3007.304604,859.001991,1,1,0,0,0,0,...,66.5739,109.6136,89.9224,63.3547,108.6231,95.6459,1,1.0,2002-1231-1428,0.532416


In [65]:
outcome = df.drop(df.columns[0:68],axis = 1)

In [66]:
outcome

,game_id,probability
0,2002-1373-1108,0.524246
1,2002-1104-1194,0.856532
2,2002-1112-1364,0.251453
3,2002-1181-1457,0.083073
4,2002-1231-1428,0.532416
5,2002-1242-1221,0.078529
6,2002-1245-1329,0.568465
7,2002-1246-1434,0.638212
8,2002-1281-1274,0.218688
9,2002-1323-1150,0.594896


In [67]:
outcome.to_csv('result_logistic.csv')

In [ ]:
#model = tree.DecisionTreeClassifier()
#model = tree.DecisionTreeRegressor()
#model = linear_model.LogisticRegression()
#model = linear_model.BayesianRidge()
#model = linear_model.Lasso()
#model = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.1)
#model = svm.SVC()
#model = svm.SVR()
#model = linear_model.Ridge(alpha = 0.5)
#model = neuralNetwork(10)
#model = AdaBoostClassifier(n_estimators=100)
#model = RandomForestClassifier(n_estimators=64)
#model = KNeighborsClassifier(n_neighbors=39)
#model = VotingClassifier(estimators=[('GBR', model1), ('BR', model2), ('KNN', model3)], voting='soft')
#model = GradientBoostingClassifier(n_estimators=100)
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5)